In [33]:
import pythainlp
import numpy as np
import math
import nltk
import re
import pandas as pd 
import seaborn as sns
import random 

from pythainlp import word_tokenize
from pythainlp import corpus
from pythainlp.corpus import thai_stopwords
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error, r2_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn import svm

nltk.download('words')
th_stop = thai_stopwords()
p_stemmer = PorterStemmer()

def split_word(text):
    tokens = word_tokenize(text,engine='icu')
    
    # Remove stop words ภาษาไทย และอักขระพิเศษ
    tokens = [i for i in tokens if not i in th_stop and not i in punctuation and not i in ['“','”','‘','’']] 
    
    # Thai transform to root word
    # tokens_temp=[]
    # for i in tokens:
    #     w_syn = wordnet.synsets(i)
    #     if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
    #         tokens_temp.append(w_syn[0].lemma_names('tha')[0])
    #     else:
    #         tokens_temp.append(i)
    
    # tokens = tokens_temp
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

def find_cosine_similarity(original_text_vector, headline_news_vector, first_text_index, second_text_index):
    sum = 0 #upper text of cosine similarity
    sum_a = 0
    sum_b = 0
    total_words_number = original_text_and_headline_news_vector.shape[1]

    for index in range(total_words_number):
        if (original_text_vector[first_text_index, index] == 0) and (headline_news_vector[second_text_index, index] ==0):
            continue
        sum_a += original_text_vector[first_text_index, index]**2
        sum_b += headline_news_vector[second_text_index, index]**2

        sum += original_text_vector[first_text_index, index]*headline_news_vector[second_text_index, index]

    # print('This is sum (upperpart) :',sum)
    
    squareroot_sum_a = math.sqrt(sum_a)
    squareroot_sum_b = math.sqrt(sum_b)
    lowerpart = squareroot_sum_a * squareroot_sum_b

    # print('This is lowerpart :',lowerpart)
    return (sum/lowerpart)

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [34]:
original_text_and_headline_news = []

clean_text = ["น้ำจะท่วมกรุงเทพฯ ในเดือนตุลา 64","โดยสรุปก็คึอ ถ้าดูจากตัวเลขปริมาณการไหลของน้ำ การระบายน้ำ ปริมาณน้ำในเขื่อนใหญ่ที่ยังน้อยมาก และยังมีพื้นที่รับน้ำได้อีกมาก รวมทั้งพื้นที่รับน้ำที่ท่วมอยู่ในเวลานี้ด้วย ก็จะเห็นว่า กรุงเทพมหานคร และนนทบุรี ยังคงไม่มีแนวโน้มจะถูกน้ำท่วม (ที่ไม่ใช่น้ำท่วมขังเวลาฝนตก)", "น้ำท่วมจะไปไหน ข่าวลือเริ่มแรง กรุงเทพฯ-นนทบุรี จะจมน้ำ?", "แหล่งข่าวผู้เชี่ยวชาญด้านทรัพยากรน้ำระบุ สถานการณ์น้ำในปัจจุบัน จากข้อมูลพบว่ายังคงแตกต่างจากปี 2554 หลังเกิดความกังวล ส่งต่อข้อมูลและเปรียบเทียบว่า น้ำจะท่วมกรุงเทพมหานครและนนทบุรีแบบมหาอุทกภัย"]
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ["ขณะนี้ คิวในการยื่นขอ VISA ประเทศสหรัฐอเมริกา คิวอยู่ที่เดือนสิงหาคม พ.ศ. 2565 แล้ว จริงหรือไม่","ระยะเวลารอคิวการยื่นวีซ่าสามารถเช็กได้ที่เว็บไซต์https://travel.state.gov/content/travel/⋯sa-information-resources/wait-times.htmlจากการเช็กล่าสุดพบว่าระยะเวลาการรอวีซ่าคือVisitor Visa 35 Calendar DaysStudent/Exchange Visitor Visas 2 Calendar DaysAll Other Nonimmigrant Visas 2 Calendar Daysไม่ถึงปี 65 แต่อย่างใด"]
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[1])

clean_text = ["ประเทศไทย มีจังหวัดเพิ่มอีก 6 จังหวัด จริงหรือ","ผู้สื่อข่าวรายงานว่า กรมการปกครอง กระทรวงมหาดไทย เผยแพร่ข้อมูลว่าตามที่ปรากฏข่าวสารผ่านสื่อออนไลน์ว่า “ประกาศชาวไทย จะมี 83 จังหวัดแล้ว” นั้น กรมการปกครอง กระทรวงมหาดไทย ชี้แจงว่า ข้อมูลข่าวดังกล่าวไม่เป็นความจริงแต่ประการใด และเป็นการแชร์ข่าวที่เคยเป็นข่าวปลอม มาตั้งแต่ปี พ.ศ.2558 แล้ว","มท.ยืนยันประเทศไทยมีแค่ 76 จังหวัด","กรมการปกครอง ร่อนหนังสือชี้แจงข่าวปลอมในโซเชียล อ้างประกาศชาวไทยจะมี 83 จังหวัด ชี้เป็นข่าวปลอมตั้งแต่ปี 2558 ยืนยันไทยมี 76 จังหวัด"]
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ["ยะลา คุมเข้มมาตรการ เคอร์ฟิว - ร้านขายอาหาร เครื่องดื่ม ปิด 2 ทุ่ม", "นายอำเภอเมืองยะลา นำทีมลงพื้นที่ กำชับมาตรการ เคอร์ฟิว - ร้านขายอาหาร เครื่องดื่ม ปิด 2 ทุ่มเคร่งครัด เพื่อลดความเสี่ยงแพร่เชื้อโควิด-19 ที่แพร่ระบาดอย่างรวดเร็ว รวมถึง คลัสเตอร์ใหญ่ในพื้นที่จังหวัดยะลา (โรงเรียน โรงงานยาง ยะลา เบตง) มียอดผู้ติดเชื้อเพิ่มสูงขึ้นอย่างต่อเนื่อง", "ยะลา คุมเข้มมาตรการ เคอร์ฟิว - ร้านขายอาหาร เครื่องดื่ม ปิด 2 ทุ่ม ลดเสี่ยงโควิด-19","นายอำเภอเมืองยะลา นำทีมลงพื้นที่ กำชับมาตรการ เคอร์ฟิว - ร้านขายอาหาร เครื่องดื่ม ปิด 2 ทุ่มเคร่งครัด เพื่อลดความเสี่ยงแพร่เชื้อโควิด-19"]
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ["ถ้าใครมีเส้นเลือดในสมองเริ่มตีบแล้ว จะไม่สามารถยืนด้วยขาข้างเดียวถึง 20 วินาที ซึ่งน่าจะจริง เพราะ ได้ไปทำ MRI ที่สมองแล้ว แพทย์พบว่า มีเส้นเลือดในสมองตีบ และ ได้มาพบคลิบนี้ ลองยืนขาเดียว ปรากฏว่า ยืนได้เพียง 3 วินาทีเท่านั้นจึงคิดว่า มีความเชื่อมโยงกันจริง ขอเชิญแต่ละท่านทดสอบว่า เส้นเลือดในสมองของท่านยังปกติ ดีหรือไม่", "ศูนย์ต่อต้านข่าวปลอม ประเทศไทย เปิดเผยว่า ตามที่มีคลิปวิดีโอแนะนำ เกี่ยวกับประเด็นเรื่อง ยืนขาเดียว ทดสอบการเป็นโรคหลอดเลือดสมองตีบ ด้วยตนเอง ทางศูนย์ต่อต้านข่าวปลอมได้ดำเนินการตรวจสอบข้อเท็จจริงโดย สถาบันประสาทวิทยา กรมการแพทย์ กระทรวงสาธารณสุข", "ข่าวบิดเบือน! ยืนขาเดียว ทดสอบการเป็นโรคหลอดเลือดสมองตีบ ด้วยตนเอง","จากประเด็นยืนขาเดียว ทดสอบการเป็นโรคหลอดเลือดสมองตีบ ด้วยตนเอง ทางสถาบันประสาทวิทยา กรมการแพทย์ ชี้แจงว่าการยืนขาเดียวได้ไม่เกิน 20 วินาที ไม่ได้เป็นการยืนยันว่าเป็นโรคหลอดเลือดสมอง ซึ่งมีโรคอื่นที่เกิดจากการทรงตัวผิดปกติได้อีก เช่นโรคของการรับความรู้สึกของข้อผิดปกติ โรคข้อต่างๆ ก็อาจจะให้ผลเช่นกัน โดยโรคหลอดเลือดสมอง มักจะมีอาการเป็นฉับพลันทันใด ต้องได้รับการรักษาที่ถูกต้องเหมาะสมอย่างทันท่วงที"]
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ["กรุงเทพธุรกิจ ลงข่าว จับตาพายุดีเปรสชั่น มินเดลเล หวั่นถล่มไทย ฝนตกหนักช่วง 24-27 กันยายน นี้(ภายหลังแก้ไขแล้ว)ส.ส.กิตติ์ธัญญา วาจาดี โพสต์เฟสบุคส์ว่า พายุดีเปรสชั่น มินเดึลเล ก่อตัวขึ้นในทะเลจีนใต้ เป็นพายุลูกที่ 16 ของปีนี้ ทำให้ฝนตกหนักถึงหนักมากที่อุบลราชธานี และพื้นที่ใกล้เคียง(ภายหลังลบโพสต์แล้ว แต่มีการแคปหน้าจอส่งกันทางไลน์)ไม่จริง เพราะพายุ มินเดึลเล อยู่แถวฟิลิปปินส์ มุ่งหน้าญี่ปุ่น ไม่สามารถข้ามมาเมืองไทยได้พายุที่เข้ามาชื่อ เตี้ยนหมู่ และได้อ่อนกำลังลงแล้ว","พายุไต้ฝุ่น 'มินเดึลเล' เคลื่อนที่มุ่งหน้าไปทางญี่ปุ่น และพายุไซโคลนเคลื่อนที่มุ่งหน้าไปทางอินเดีย","เปิดเส้นทางพายุ 3 ลูก 'เตี้ยนหมู่ มินเดึลเล ไซโคลน' ไทยโดนด้วย อีสาน-กลาง ฝนตกหนัก",'ประธานกรรมการบริหาร "ทีมกรุ๊ป" หรือ TEAMG เปิดเส้นทางพายุ 3 ลูก "พายุเตี้ยนหมู่" พายุไซโคลน และ พายุดีเปรสชั่น "มินเดึลเล" ไทยมีฝนตกหนักในภาคอีสาน ภาคกลาง']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['"จิงจูฉ่าย" เป็นผัก มหัศจรรย์ สามารถใช้รักษามะเร็งได้ !————————————“ผม..นิกกี้ อิทธิเกษม KU37 อดีตนายกสมาคมม.เกษตรฯแห่งอเมริกา จะขอเล่าประสบการณ์ที่เคยสัมผัสมากับตัวเองเมื่อปี 2007 เพื่อเป็นcaseตัวอย่างและเป็นทางออกให้กับผู้ป่วยโรคมะเร็งทั้งหลาย       ผมมีร้านอาหารอยู่ในเขต N.Hooywood วันหนึ่งกลางปี 2007 ผมได้มีโอกาสเจอลูกศิษย์เก่าที่เคยมาเรียนแจกไพ่ที่ร้าน ถามทุกข์สุขกันไปมา ถึงได้รู้ว่าเขาเคยเป็นมะเร็งที่คอ 1 ครั้ง,มะเร็งที่ต่อมลูกหมาก 1 ครั้ง ปัจจุบันหายเป็นปลิดทิ้ง รวมทั้งพี่น้องญาติอีก8คนที่เป็นมะเร็งภายในเวลา5 ปี ทุกคนหายหมด ผมจึงเกิดความกระตือรือร้นที่จะถามว่า มีวิธีรักษากันอย่างไร? วันนั้นคุณแม่เขามาด้วยจึงอธิบายให้ฟังว่า "ฉันมีต้นจิงจูฉ่ายอยู่ในสวนหลังบ้านเยอะ ปกติก็จะใส่ในแกงจืดให้ลูกๆกินเป็นประจำ เพราะคนจีนบอกต่อๆกันมาถึงสรรพคุณช่วยฟอกเลือดและขับสารพิษออกจากร่างกายทำ ให้ลูกๆไม่ค่อยเป็นอะไรวันหนึ่งลูกฉันไปตรวจหมอกลับมา บอกว่าเป็นมะเร็ง อีก 2 เดือนต้องไปฉายแสง ฉันจึงลองเอาจิงจูฉ่าย1กำมือมาตำ จะได้น้ำออกมาแก้ว(เล็ก)หนึ่ง ให้เขากินตอนเช้าทุกวัน 2 เดือนถัดมาไปตรวจหมอ หมอถามไปทำอะไรมา ทุกอย่างปกติหมด มะเร็งได้หายไปแล้ว"        หลังจากคุยกันพักหนึ่ง ผมก็ได้ข้อสรุปออกมาว่า คน8คนเป็นมะเร็งในที่ต่างกัน กินน้ำจิงจูฮวยฉ่าย 2-3 เดือน ทุกคนหายหมด(ไม่มีใครต้องทำคีโมเลย) ผมจึงเกิดความเชื่อถือขึ้นมาระดับหนึ่ง        ก่อนหน้านั้น พี่สะใภ้ผม 2 คน (ปี2006) เป็นมะเร็งที่เต้านม,มดลูก คนหนึ่งต้องทำคีโม ใช้วิธี"นั่งสมาธิ"ทุกวัน 8 เดือน หายเป็นปกติ        ต่อมาผมได้ต้นจิงจูฉ่ายมา1 ต้น จากคุณแม่ของลูกศิษย์ จึงรีบนำมาเพาะขยาย ปลายปี 2008 ผมได้รู้ว่าแม่ค้าที่เช่าที่ในร้านผมคนหนึ่งเป็นมะเร็งเต้านมขั้น 3 หน้าคล้ำ เดินตัวแข็งแล้ว ผมรีบเชิญมานั่งคุยเล่าเรื่องจิงจูฉ่ายให้ฟังและเรื่องพี่สะใภ้นั่งสมาธิ ก็เลยแนะนำให้ทำ 2 อย่างควบคู่กันไป เชื่อไหมว่า 3 เดือนถัดไป หายเป็นปลิดทิ้ง (ผมให้เขาไปเพียง1ต้นไปปลูกและ เด็ดใบทานเลยวันละ1ใบ เพราะมีน้อย ก็ยังได้ผล)        ปี 2009 ผมมีต้นจิงจูฉ่ายมากขึ้นและแจกจ่ายให้กับคนรู้จักหลายคน ซึ่งกำลังอยู่ในระหว่าง 3 เดือน ทุกคนอาการดีขึ้นหมด ถ้าเราอยากช่วยผู้ที่เรารักและห่วงใยซึ่งกำลังเผชิญกับโรคร้ายนี้ ผมแนะนำวิธีนี้ ถ้าคุณต้องทำคีโมก็ทำไป กินใบจิงจูฉ่ายและนั่งสมาธิผมว่าคุณมีโอกาสหาย        ผมพยายามหาข้อมูลเพิ่มเติม พบว่าผักชนิดนี้ แพมย์จีนโบราณเรียกว่า"ยาเย็น"เป็น"หยิน"ช่วยฟอกเลือด,ขับสารพิษ..ฆ่าเชื้อ ไวรัสทุกชนิด..เพื่อนku37 คนหนึ่งลองให้ลูกกิน เนื้องอกที่คอก็ยุบลง      เนื่องจากที่ LA.เรามีน้อยมาก ถ้าเกิดใครต้องการมากๆ (จริงๆต้องทานวันละ1กำ)ลองคุยกับคุณน้อยที่สวนคุณน้อย จ.เชียงใหมเขาขายกก.ละ 35 บาท     ใครเป็นมะเร็งลองดูซิครับ ทานแล้วยังไงก็ไม่มีผลข้างเคียง แต่ คุณมีโอกาสหายครับ.ใครที่เคยสั่ง "เกาเหลาเลือดหมู" มาทาน เคยสงสัยกันไหมว่า ในชามเกาเหลาของเราจะมีผักสีเขียวชนิดหนึ่ง ที่ไม่ใช่ผักกาดหอม ขึ้นฉ่าย หรือใบตำลึงใส่ชามมาด้วย หลายคนไม่ทราบว่าเจ้าผักชนิดนี้มีชื่อเรียกว่าอะไร แล้วมีสรรพคุณอย่างไร เอ้า...ใครที่ไม่รู้ ตามกระปุกดอทคอมมาเลยค่ะ         เจ้าผักชนิดนี้เรียกว่า "จิงจูฉ่าย" ค่ะ หรือที่ชาวต่างชาติเรียกว่า "เซเลอรี่" (Celery) เป็นผักสมุนไพรชนิดหนึ่งของจีน มีชื่อวิทยาศาสตร์ว่า Apium graveolens L. ลักษณะต้น "จิงจูฉ่าย" จะเป็นกอคล้ายใบบัวบก สามารถเจริญงอกงามได้ดีในที่ที่มีแสงแดดรำไร ชื้น ดินโปร่งแต่ไม่แฉะ ชอบอากาศเย็นมากกว่าอากาศร้อน คุณค่าทางโภชนาการของ "จิงจูฉ่าย" มีไม่น้อยทีเดียวค่ะ เพราะ "จิงจูฉ่าย" 100 กรัม ให้พลังงาน 392 กิโลแคลอรี ประกอบด้วยสารอาหารนานาชนิด คือ โปรตีน, ไขมัน, คาร์โบไฮเดรต, เส้นใย, แคลเซียม, เหล็ก, ฟอสฟอรัส, วิตามินเอ, วิตามินบี6, วิตามินซี และวิตามินอี         มาที่สรรพคุณทางยากันบ้างดีกว่า จุดเด่นของ "จิงจูฉ่าย" คือมีกลิ่นหอม คล้าย ๆ กับตั้งโอ๋ ยิ่งโดนความร้อนจะยิ่งหอม และยิ่งเพิ่มสรรพคุณมากขึ้น โดยกลิ่นหอมของ "จิงจูฉ่าย" มาจากน้ำมันหอมระเหยที่มีอยู่ในลำต้นและใบนั่นเอง ประกอบด้วยสารไลโมนีน ซิลนีน และสารกลัยโคไซด์ที่มีชื่อว่า อะปิอิน ซึ่ง สารเหล่านี้มีสรรพคุณช่วยปรับสมดุลความดันโลหิต จึงเหมาะกับผู้ที่มีปัญหาเรื่องความดัน แถมยังช่วยขับลมในกระเพาะอาหารและลำไส้ได้ด้วยส่วนต้นสด และเมล็ดของ "จิงจูฉ่าย" มีโซเดียมต่ำ จึงดีต่อผู้ป่วยโรคไต          นอกจากนี้ ในทางการแพทย์เชื่อว่า "จิงจูฉ่าย" เป็นยาเย็น จึงช่วยบำรุงปอด ช่วยฟอกเลือด เลือดลมหมุนเวียนได้สะดวก คนจีนจึงนิยมนำผักชนิดนี้มาปรุงเป็นอาหารรับประทานในหน้าหนาว เพื่อช่วยในเรื่องการไหลเวียนของโลหิต ปรับสมดุลให้ร่างกายได้ดีนั่นเอง ส่วนที่หลายคนสงสัยว่า ทำไมเราจึงมักเห็น "จิงจูฉ่าย" อยู่ในเกาเหลาเลือดหมู นั่นก็เพราะ "จิงจูฉ่าย" มีสรรพคุณช่วยดับกลิ่นคาวเลือดได้ดีด้วยค่ะแต่จริง ๆ แล้ว "จิงจูฉ่าย" ไม่ได้ใช้ทำอาหารได้เพียงแค่ต้มเลือดหมูเท่านั้นนะ เพราะอาหารประเภทแกงจืดทั้งหลาย หรือผัดผัก ผัดฉ่าก็สามารถใช้ "จิงจูฉ่าย" เป็นส่วนผสมที่ลงตัวน่ารับประทานไม่แพ้กันจิงจูฉ่าย ในรูปแบบชา  คัดวัตถุดิบใบจิงจูฉ่ายคุณภาพ  ปลอดสาร ผ่านกระบวนการผลิตที่ได้มาตรฐาน สะอาดปลอดภัย ไม่มีสารปนเปื้อน รสชาติหอมหวานไม่ขม ดื่มง่าย โดยกระบวนการหมัก นวดอย่างพิถีพิถัน และอบด้วยไฟอ่อนเพื่อให้คงคุณค่าทางโภชนาการ และสรรพคุณทางยา  ชาจิงจูฉ่ายจึงมีประโยชน์มากมาย  และเนื่องจากเป็๋นสมุนไพรจึงไม่มีผลข้างเคียงในกรณีที่บริโภคเป็นปริมาณมากอยกได้ข้อมูลเพิ่มเติมจากเพื่อนในเวปค่ะLiked By: PinKenมะเร็งระยะสุดท้าย 2 ปี ไม่เข้ารับรักษา ค่ามะเร็งล…: https://youtu.be/uITAm2Gi5mQ','ผักจิงจูฉ่ายรักษามะเร็งระยะสุดท้ายทำให้ค่ามะเร็งลดลง กรมการแพทย์ โดย สถาบันมะเร็งแห่งชาติ ได้ดำเนินการตรวจสอบข้อเท็จจริงพบว่า น้ำปั่นผักจิงจูฉ่ายไม่สามารถนำมาใช้รักษามะเร็งระยะสุดท้ายในมนุษย์ได้','เตือน!!ข่าวปลอม “น้ำปั่นผักจิงจูฉ่าย"รักษามะเร็งระยะสุดท้าย','สถาบันมะเร็งแห่งชาติ แจ้งเตือนข่าวปลอม “น้ำปั่นผักจิงจูฉ่ายรักษามะเร็งระยะสุดท้าย" ยืนยันไม่สามารถนำมาใช้รักษามะเร็งระยะสุดท้ายในมนุษย์ได้']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['ในที่สุด ความจริงก็ถูกเปิดเผย โดย ผู้ตัดต่อพันธุกรรมเชื้อโควิด 19 เอง...**************โควิด19 มาจากฝีมือมนุษย์มีแหล่งที่มาจากห้องแลป ไวรัส P3รัฐคาโรไลน่าเหนือของอเมริกา!!!นาย Greg Roubini ผู้เชี่ยวชาญด้านข่าวกรองชื่อดังของอเมริกาให้สัมภาษณ์ของผู้สื่อข่าวช่องข่าวทีวีที่1 ของอเมริกาได้เป็นผู้เผยความลับนี้นาย Greg เผยว่า ไวรัสโควิด19 ได้รับการออกแบบทางพันธุกรรมเพื่อใช้เป็นอาวุธชีวภาพ- มีแหล่งที่มาจากห้องแลป BSL-3 รัฐ คาโรไลน่าเหนือ พัฒนาโดย ศาสตราจารย์ราล์ฟ บาร์ริก- พร้อมกันนั้น เขาระบุว่า ไวรัสถูก “รัฐบาลมืด” จากรัฐคาโรไลน่าเหนือส่งไปแพร่ระบาดในประเทศจีน อิตาลี และอเมริกาทั้งประเทศ##..ก่อนหน้านี้ในวันที่ 15 มีนาคม นายเกรก ก็ได้ทวิตข้อความถามนายทรัมป์ว่า- เหตุใดจึงไม่บอกประชาชนอเมริกาว่า ไวรัสผลิตจากอเมริกา? ทำไมไม่อธิบายให้ชัดเจนว่าตัวไวรัสเองแท้จริงแล้วคืออาวุธชีวภาพ?**บังเอิญ ศาสตราจารย์ Luc Montanier ผู้ได้รับรางวัลโนเบลเนื่องจากเป็นผู้ค้นพบไวรัสเอชไอวีได้เปิดเผยกับนักข่าวชาวฝรั่งเศสเมื่อไม่กี่วันที่ผ่านมาว่า- โควิด19 ไม่ใช่มาจากธรรมชาติ หากแต่ได้รับการพัฒนาอย่างประณีตโดยนักวิทยาศาสตร์ชีวโมเลกุล***ศาสตราจารย์ Luc Montanier ยืนยันว่า เป็นเรื่องเด่นชัดที่เจ้าหน้าที่ผู้เชี่ยวชาญได้นำเชื้อไวรัสที่มาจากค้างคาวเข้าไปเพิ่มความเข้มข้นของเชื้อเอชไอวีเข้าไปด้วย- นี่คือ การวางยาพิษที่ชั่วร้ายที่สุดที่ไม่เคยมีมาก่อนในประวัติศาสตร์ของโลก***นั่นคือการแพร่ระบาดของไวรัสโควิด19 สุดโหด ข่าวเกี่ยวกับ “เชื้อโควิด19 เป็นอาวุธชีวภาพที่มาจากการตัดต่อพันธุกรรมโดยฝีมือมนุษย์” มาโดยตลอด***นักวิทยาศาสตร์ทั่วโลกพยายามทำงานหาแหล่งที่มาของเชื้อไวรัสโดยนักวิทยาศาสตร์อินเดียค้นพบว่า เชื้อไวรัสโคโรน่าสายพันธ์ุใหม่มีเชื้อเอชไอวีแทรกอยู่ด้วย นี่แสดงให้เห็นว่าไวรัสตัวนี้มาจากการตัดต่อทางพันธุกรรม***กลางเดือนมีนาคม นักวิทยาศาสตร์ได้วิเคราะห์พบว่าเชื้อไวรัสโควิด19จากผู้ป่วยรายหนึ่งในรัฐวอชิงตันพบว่าวัฏจักรวิวัฒนาการของมันมียาวนานกว่าครึ่งปีมาแล้ว พร้อมๆกับการศึกษาลึกซึ้งลงไปว่า ประเทศต่างๆในโลกไม่น้อยได้เบนสายตาแห่งความสงสัยไปที่อเมริกา ประเทศต่างๆ ทั้งญี่ปุ่น อิตาลี ออสเตรเลีย ล้วนมีผู้ป่วยทียืนยันว่ามีแหล่งที่มาจากอเมริกาทั้งสิ้น*** ในเวลาต่อมา ROBERT REDFIELD ผู้อำนวยการศูนย์ควบคุมและป้องกันโรคแห่งสหรัฐอเมริกายอมรับว่า ผู้ป่วยตายจากไข้หวัดใหญ่ในเดือนกันยายน 2019 มีอยู่ไม่น้อยที่ตายจากเชื้อไวรัสโควิด19 นี้- ต่อปัญหานี้โฆษกกระทรวงต่างประเทศของจีน นายจ้าวลี่เจียงได้ทวิตข้อความในทวิตเตอร์ถามผู้อำนวยการศูนย์ควบคุมและป้องกันโรคแห่งสหรัฐอเมริกา ว่าผู้ป่วยรายแรกของอเมริกาเกิดขึ้นตอนไหน? ชื่ออะไร? อยู่โรงพยาบาลอะไร? และเป็นไปได้อย่างมากที่ทหารอเมริกานำเชื้อมาแพร่ที่อู่ฮั่น.>>>>อเมริกาต้องโปร่งใส ต้องเปิดเผยข้อมูลนี้ให้โลกได้รู้ความจริง**ด้วยความพยายามอย่างสุดความสามารถของคณะผู้สื่อข่าวคณะหนึ่งแห่งรัฐเวอร์จิเนีย ในที่สุดก็ได้ตามหาผู้ป่วยรายแรกจนพบ นั่นก็คือ ทหารอเมริกาที่เข้าร่วมการแข่งขันกีฬาทหารที่อู่ฮั่นของจีนในเดือนตุลาคม 2019 นางมีชื่อว่า "Maatje Benassi">>>นายทหารหญิงของอเมริกาคนนี้มีภูมิหลังพิเศษตรงที่นางมีความเกี่ยวข้องกับห้องปฏิบัติการชีวเคมี P4 ของนาย FORT DETRICK*** คนในครอบครัวก็มีหลายคนที่ยืนยันว่าผู้ติดเชื้อในจำนวนนี้มีอยู่คนหนึ่งเป็นผู้ป่วยที่ติดเชื้อรายแรกในฮอลแลนด์ ก่อนติดเชื้อเขาเคยไปในเขตพื้นที่ลอมบาร์เดียของอิตาลี ทำให้เขตพื้นที่นั้นเกิดการแพร่ระบาดของเชื้อโควิด19***มาถึงตรงนี้ หลักฐานเกี่ยวกับเชื้อไวรัสโควิด19 มีต้นกำเนิดจากอเมริกาอย่างแน่นอน มีห่วงโซ่เชื่อมร้อยอย่างครบถ้วน ทหารพิเศษ 5 คนที่อเมริกาส่งเครื่องบินมารับกลับไปภายหลังการแพร่ระบาดของไวรัสและห้องแลปที่ถูกปิดตาย ก็สามารถนำมาปะติดปะต่อกันได้แล้วหากว่ากันตามตรรกะของนายทรัมป์ เราก็สามารถเรียกเชื้อโควิด19 เป็น "ไวรัสนอร์ธคาโรไลนา" (Virus North Carolina) หรือ "ไวรัสอเมริกา"***ในขณะที่หลักฐานทั้งหมดต่างชี้ไปที่อเมริกาเจ้าหน้าที่ชั้นสูงของอเมริกายอมรับอย่างเปิดเผยว่า เชื้อโควิด19 ไม่จัดอยู่ในชั้นของโรคระบาด แต่จัดอยู่ในชั้นของอาวุธชีวภาพ>>>#”ความไร้ยางอายทำให้โลกตะลึงและได้เพิ่มข้อน่าสงสัยว่าอเมริกาเป็นผู้วางยาพิษคนทั้งโลก. เพื่อขายวัคซีนป้องกันมาแก้ปัญหาเศรษฐกิจที่ขาดดุลการค้า”>>>เรื่องทั้งหมดได้ปรากฏชัดเจนแล้ว แต่ทว่าทรัมป์ยังพยายามโยนบาปอย่างไม่คิดชีวิตให้จีนรับเคราะห์แทนอย่าง น่ารังเกลียดที่สุด***เชื้อโควิด19 ได้ก่อให้เกิดภัยพิบัติและความสูญเสียที่ยากจะประเมินได้ บาปนี้มันใหญ่หลวงเกินกว่าจะโยนออกไปแล้วโทษคนอื่น***ยังมีข้อน่าสงสัยที่นายเกรกได้ตีแผ่ออกมา นายราล์ฟ บาร์ริค ผู้รับผิดชอบพัฒนาไวรัส รัฐคาโรไลนาเหนือคนนี้เป็นใคร*** นาย บาร์ริคมาจากมหาวิทยาลัยคาโรไลนาเหนือ เขาเป็นหัวหน้านักไวรัสวิทยาที่เปลี่ยนโฉมใหม่ของโรคซาร์สโคโรนาไวรัสโดยการตัดต่อยีนในปี 2015- และเขายังเป็นผู้นำในการวิจัยและพัฒนาไวรัสดังกล่าวอีกด้วย ที่น่าตกใจก็คือ เขาเป็นบุคคลที่รับผิดชอบด้านการพัฒนาทางคลินิกของยาวิเศษ "RADEXIVIR" เป็นไปอย่างที่โบราณว่าไว้ คนที่วางยาพิษก่อนอื่นต้องเตรียม# ยาแก้พิษไว้ก่อนเสมอ!!!!- ยา RIDESIVIR ภายหลังจากปฏิบัติการทางคลินิกและถูกตั้งข้อสงสัยโดยผู้เชี่ยวชาญเกี่ยวกับประสิทธิภาพและความปลอดภัยของมันจึงทำให้ตกกระป๋องไปพร้อมๆกับการแพร่ระบาดที่ลุกลามออกไปทั่วโลก***อเมริกากลายเป็น “ศูนย์กลางการล้างโลก” ไปแล้ว- การแพร่ระบาดในช่วงแรกของอเมริกา ประธานาธิบดีทรัมป์ไม่ได้ให้ความสาคัญกับมันเลยโดยมองว่าเป็นไข้หวัดใหญ่ที่หนักกว่าปกติเท่านั้นเพราะรู้อยู่แก่ใจว่าคนของตนเองผลิตมันขึ้นมาจนกระทั่งเพื่อนรักของเขาคือ "นายสแตนลี่ย์ เชล่า" เจ้าพ่อวงการอสังหาริมทรัพย์ที่ยิ่งใหญ่แห่งนิวยอร์กเสียชีวิตจากเชื้อไวรัสโควิด19>>>>ถึงเวลานี้จีนได้ฟ้องร้องต่อศาลโลกว่า อเมริกาเป็นต้นเหตุในการแพร่เชื้อไวรัสโควิด 19 อย่างตั้งใจเพื่อทำลายล้างจีนและ ปชช ทั่วโลก***>>>ตอนนี้คงต้องรอดูการสืบสวนของศาลโลกว่าจะตัดสินออกมาเช่นไร? ซึ่งถึง ณ เวลานี้ ทรัมป์เริ่มรู้สึกตัวและให้ความสาคัญในระดับสูง #แต่ว่าสายไปเสียแล้ว!!!**Ny Ny*','อ้างว่า ดร. บาริค เป็นผู้พัฒนาเชื้อนี้เป็นอาวุธชีวภาพ แต่ความจริงแค่มีงานวิจัยศึกษาว่าเชื้อโคโรนาติดต่อจากค้างคาวมาสู่คน การความพยายามอ้างเรื่อง Fort Detrick ในทางที่ไม่มีหลักฐานสนับสนุนชัดเจน','โควิด-19 : จีนกับความพยายามแพร่ข่าวปลอมเรื่องโควิดมีต้นกำเนิดในสหรัฐฯ','การระดมเผยแพร่ข้อมูลเท็จที่กล่าวหาว่าเชื้อโรคโควิด-19 มีต้นกำเนิดมาจากฐานทัพทหารในรัฐแมริแลนด์ กำลังได้รับความสนใจอย่างมากในจีน ก่อนหน้าที่หน่วยงานด้านข่าวกรองของสหรัฐฯ จะเปิดเผยรายงานการสอบสวนเรื่องต้นกำเนิดของเชื้อไวรัสชนิดนี้']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['คลัง ขยายมาตรการช่วยค่าน้ำ ค่าไฟ ให้ผู้ถือบัตรคนจน','มาตรการบรรเทาภาระค่าไฟฟ้าและค่าน้ำประปา มีวัตถุประสงค์เพื่อบรรเทาภาระค่าใช้จ่ายด้านสาธารณูปโภคขั้นพื้นฐาน ได้แก่ ค่าไฟฟ้าและค่าน้ำประปาให้แก่ผู้มีบัตรสวัสดิการแห่งรัฐ (บัตรฯ) อย่างต่อเนื่องและครอบคลุมผู้มีบัตรฯ ให้สามารถได้รับประโยชน์จากมาตรการบรรเทาภาระค่าไฟฟ้าและค่าน้ำประปา (มาตรการฯ) ดังกล่าวให้มากขึ้น','เช็กเลย!เปิดเกณฑ์ลดค่าน้ำ-ค่าไฟผู้ถือบัตรคนจน เผยช่องทางลงทะเบียน','เช็กก่อนพลาดโอกาส! เปิดเกณฑ์ผู้ถือบัตรคนจน ได้ลดค่าน้ำ-ค่าไฟ 12 เดือน พร้อมช่องทางลงทะเบียน']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['''ช็อก! Bill Gates เรียกร้องให้ถอนวัคซีน Covid-19 ทั้งหมด "วัคซีนอันตรายกว่าที่คิด"โดย W. Gellesวอชิงตัน- (MaraviPost) - ในการประกาศที่น่าตกใจ Bill Gates มหาเศรษฐีผู้ร่วมก่อตั้ง Microsoft และกำลังสำคัญที่อยู่เบื้องหลังวัคซีน COVID-19 เรียกร้องให้มีการนำวัคซีนจากพันธุกรรม COVID-19 ทั้งหมดออกจากตลาดทันทีในการกล่าวสุนทรพจน์ทางโทรทัศน์ที่เจ็บปวดรวดร้าวบ่อยครั้ง 19 นาที เกตส์กล่าวว่า “เราทำผิดพลาดอย่างมหันต์ เราต้องการปกป้องผู้คนจากไวรัสอันตราย แต่กลับกลายเป็นว่าไวรัสมีอันตรายน้อยกว่าที่เราคิดไว้มาก และวัคซีนก็อันตรายเกินกว่าที่ใครจะจินตนาการได้”โดย W. Gelles“วัคซีนเหล่านี้—ไฟเซอร์, โมเดอร์นา, จอห์นสัน แอนด์ จอห์นสัน, แอสตร้าเซเนก้า—พวกมันกำลังฆ่าคนทั้งซ้ายและขวา—และพวกเขากำลังทำร้ายคนบางคนอย่างเลวร้ายมาก” เกตส์กล่าวต่อ โดยโบกมือขึ้นไปในอากาศเป็นครั้งคราวเพื่อให้ได้ผลลัพธ์ที่น่าทึ่ง“ข้อมูลของรัฐบาลเองแสดงให้เราเห็นว่านี่คือสิ่งที่กำลังเกิดขึ้น ระบบการรายงานของ CDC กำลังแสดงอะไร…ประมาณ 13,000 ผู้เสียชีวิตในสหรัฐอเมริกาและเหตุการณ์ไม่พึงประสงค์กว่าครึ่งล้าน เราทุกคนรู้ดีว่าระบบการรายงานเป็นเรื่องหลอกลวง“เราทราบดีว่า VAERS [ระบบการรายงานเหตุการณ์ไม่พึงประสงค์จากวัคซีนของศูนย์ควบคุมและป้องกันโรค] รวบรวมได้เพียงร้อยละหนึ่งของสิ่งที่เกิดขึ้น เรากำลังพูดถึงผู้เสียชีวิตจากวัคซีนโควิดมากกว่าล้านราย และผู้คนกว่า 60 ล้านคนที่มีผลข้างเคียงที่ไม่ดี”นี่ไม่ใช่สิ่งที่เราต้องการ สิ่งนี้ไม่เป็นที่ยอมรับ” นายเกทส์ยืนยันหุ้นในวอลล์สตรีทของบริษัทวัคซีนโควิดรายใหญ่ทั้งหมดร่วงลง 20% ถึง 30% เนื่องจากนายเกตส์ประกาศว่าเขาเข้าร่วมในคำร้องเร่งด่วนของพลเมืองที่ยื่นโดยองค์กรป้องกันสุขภาพเด็กของโรเบิร์ต เอฟ. เคนเนดี จูเนียร์ ที่เรียกร้องให้มีอาหารสหรัฐฯ และ สธ. ถอนวัคซีนโควิดทั้งหมดออกจากตลาดทันทีGates กล่าวต่อว่า “ผู้คนจำนวนมากที่รับวัคซีนเหล่านี้เสียชีวิต… หนึ่งวัน สองวัน ห้าวันหลังจากได้รับการฉีดวัคซีน คนอื่นๆ มีอาการอัมพาต ตาบอด ชัก หัวใจวาย ระบบภูมิคุ้มกันล้มเหลว ลิ่มเลือดอุดตัน สมองอักเสบ ปอดหรือไตถูกทำลาย การแท้งบุตร โรคภูมิต้านตนเอง ความล้มเหลวของระบบอวัยวะหลายส่วน ความเหนื่อยล้าถาวรอย่างถาวร และปัญหาร้ายแรงอื่นๆ อีกมากมาย“แน่นอน Media Mouthpieces ของเรา—ฉันหมายถึงสื่อกระแสหลัก ละเลยโศกนาฏกรรมเหล่านี้ทั้งหมดว่าเป็น 'แค่เรื่องบังเอิญ'”“เหตุผลที่พวกเขาพูดอย่างนั้น” เกทส์อธิบาย “เป็นเพราะสิ่งที่ฉันทำที่งาน 201 ซึ่งเป็นการจำลองสถานการณ์แพร่ระบาดของไวรัสโคโรน่าที่นิวยอร์กในเดือนตุลาคม 2019 เพียงไม่กี่สัปดาห์ก่อนที่เราจะประกาศการแพร่ระบาดที่เกิดขึ้นจริง ฉันได้รับหนังสือพิมพ์ ช่องทีวี และสถานีวิทยุรายใหญ่ทั้งหมดเพื่อตกลงที่จะปฏิบัติตามคำบรรยายอย่างเป็นทางการ—'วัคซีนมีความปลอดภัยและมีประสิทธิภาพ' และเพื่อเซ็นเซอร์ใครก็ตามที่ตั้งคำถามเกี่ยวกับ BS นี้“ดังนั้น สาธารณชนจึงไม่เคยได้ยินหลักฐานจากแพทย์และนักวิจัยทางการแพทย์ที่มีชื่อเสียงหลายร้อยคนที่เตือนว่าวัคซีนมีอันตรายและมักเป็นอันตรายถึงชีวิต”“นั่นเป็นความผิดพลาดครั้งใหญ่ของฉัน” เกทส์กล่าว ดูเหนื่อยและบางครั้งก็มีน้ำตา “เราไม่ควรทำอย่างนั้น ประชาชนมีสิทธิทุกอย่างที่จะได้รับข้อมูลข่าวสารที่ดี เพื่อรับข้อเท็จจริงทั้งหมดเพื่อที่พวกเขาจะได้ตัดสินใจอย่างมีเหตุมีผล”เปลี่ยนหัวข้อราวกับว่าต้องการแสดงความเห็นอกเห็นใจ มิสเตอร์เกทส์วางใจว่า: “ฉันผ่านช่วงเวลาที่ยากลำบากและค้นหาจิตวิญญาณมากมายตั้งแต่เมลินดาทิ้งฉัน การหย่าร้างครั้งนี้ทำให้ฉันต้องมองตัวเองให้ดี ฉันไม่ต้องการที่จะถูกจดจำในฐานะสัตว์ประหลาดที่ฆ่าคนนับล้านด้วยวัคซีนที่อันตรายถึงตาย ฉันไม่ใช่สัตว์ประหลาด ฉันไม่ใช่ฆาตกร ฉันไม่ต้องการให้ครอบครัว เพื่อน และบริษัทจำได้ว่าฉันเป็นนักฆ่าหมู่"บางคนเรียกฉันว่าเป็นนักสังคมสงเคราะห์หรือแม้แต่โรคจิตเภทเพราะแผนการณ์ของฉันในการช่วยเหลือมนุษยชาติ - เช่นการลดภาวะโลกร้อนด้วยการพ่นฝุ่นสู่บรรยากาศชั้นบนหรือปล่อยยุงดัดแปลงพันธุกรรมหลายล้านตัวเพื่อต่อสู้กับไข้เลือดออกและไวรัสซิกา''', '''The quint รายงานว่า ข่าวปลอมนี้ถูกแชร์ไปหลายสื่อโซเชียล ทั้งเฟซบุ๊ก ทวิตเตอร์ รวมทั้ง WhatsApp เพราะผู้ใช้เชื่อว่าเป็นความจริง และก้อปปี้เนื้อหาจากข่าวปลอมนี้ไปแชร์ ทั้งๆที่เกตส์เป็นหัวหอกในการวางแผนวัคซีนโควิด เมื่อพบว่ามีการแชร์ข่าวปลอมนี้ออกไป The quint ได้ เสริช์คีย์เวิร์ด จากกูเกิลเพื่อค้นหา เว็บไซต์ Daily Expose ซึ่งได้พบว่ามีบทความหลายชิ้นในเว็บไซต์ที่ตรวจสอบข้อเท็จจริงอย่าง Full Fact ได้หักล้างบทความหลายชิ้นที่ Daily Expose นำเสนอและเมื่อเช็คลิ้งค์ของข่าวนี้พบว่ามีการปรับพาดหัวข่าวใหม่ ว่าเป็นรายงานเชิงเสียดสี''', '''เช็คก่อนแชร์ ข่าวปลอม”บิล เกตส์เรียกร้องให้ถอนวัคซีนโควิด”''','''การระบาดของไวรัสโควิด-19 ส่งผลให้ประชาชนตื่นตัวและหาข้อมูลความรู้เกี่ยวกับไวรัส การฉีดวัคซีน เพื่อเป็นแนวทางการดูแลตัวและป้องกันโรคมากขึ้น ซึ่งในขณะเดียวกันก็มีผลให้เกิดการสร้างข้อมูลปลอม การสร้างข่าวเทียมขึ้นมามากมาย มีประชาชนบางกลุ่มหลงเชื่อจนส่งต่อและแพร่กระจายในโลกโซเชียลอย่างรวดเร็วไม่แพ้การกระจายของไวรัสเลยทีเดียว''']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['มีกันแชร์ในโลกโซเชี่ยลว่า มีผู้ติดโควิด เป็นคลัสเตอร์ใหม่ จำนวน 50 คน ที่ต.ตาลสุม อ.ตาลสุม จ.อุบลราชธานี จริงหรือไม่','เป็นกลุ่มการพนัน เพียงการตรวจเบื้องต้น อยู่ระหว่างรอสอบสวนโรค รอการตรวจ PCR ยืนยันอีกครั้ง 15.00 น .21-09-64 ข้อมูล : สำนักงานสาธารณสุขจังหวัดอุบลราชธานี']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[1])

clean_text = ['เปิดสถานศึกษา','ศธ.จ่อเปิดเรียนอำเภอปลอดโควิด 29 จังหวัดพื้นที่สีแดงเข้ม มั่นใจฉีดวัคซีนเด็กไร้ปัญหา ครูฉีดแล้วกว่า 70%','กระทรวงศึกษาธิการ เตรียมเปิดสถานศึกษาในพื้นที่ 29 จังหวัดสีแดงเข้ม','นางเกศทิพย์ ศุภวานิช ผู้ตรวจราชการกระทรวงศึกษาธิการ (ศธ.) ในฐานะโฆษกกระทรวงศึกษาธิการ  เปิดเผยว่า กระทรวงศึกษาธิการ ได้หารือกับกรมอนามัย กระทรวงสาธารณสุข (สธ.) เพื่อวางแนวทางกระจายวัคซีน โดยจะส่งไปที่จังหวัดเพื่อส่งต่อไปยังโรงเรียน   เมื่อวัคซีนพร้อมก็สามารถดำเนินการฉีดได้ทันที ส่วนการฉีดวัคซีนครูนั้น ดำเนินการไปแล้วกว่า 70% เหลืออีก 30%  คิดว่าไม่น่าจะมีปัญหา']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['เพิ่งรู้นะเนี่ย วิธีเลือกใช้ยาสีฟันให้ดูที่แถบท้ายหลอดยาสีฟันครับ สีแดงผลิตมาจากสารเคมีผสมธรรมชาติ,สีดำผลิตจากสารเคมีล้วนๆ,สีน้ำเงินผลิตมาจากธรรมชาติผสมตัวยาของแพทย์,สีเขียวผลิตจากธรรมชาติครับ','ศูนย์ชัวร์ก่อนแชร์รายงานว่า ปลายหลอดที่ระบุตามสีนั้น เพื่อให้เซนเซอร์เครื่องจักรในการตรวจจับ ก้นหลอดยาสีฟัน ไม่เกี่ยวข้องกับวัตถุดิบ หรือสารประกอบในยาสีฟันแต่อย่างใด หากต้องการทราบส่วนผสมของยาสีฟัน เราสามารถศึกษารายละเอียดได้จากข้างหลอดยาสีฟันที่มีการพิมพ์บอกไว้','แถบสีปลายหลอดยาสีฟัน ไม่ใช่ตัวบ่งชี้ชนิดสารเคมี','ความจริงของแถบสีบนหลอดยาสีฟัน! เป็นคนละเรื่องกับข้อมูลผิดๆ ที่มีการส่งต่อกันในโลกโซเชียลที่บอกว่า แถบสีต่างๆ ที่ก้นหลอดยาสีฟันคือตัวบ่งชี้ว่ายาสีฟันนั้นทำมาจากสารเคมีหรือวัสดุธรรมชาติ ทำให้ผู้รับข่าวสารเกิดความเข้าผิดในวงกว้าง']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['ประกาศ เตือนภัย มิจฉาชีพแฝงตัวในสื่อออนไลน์ช่องทางต่างๆ โปรดระวัง เฟซบุ๊ก บัญชีไลน์ หรือ อีเมล ปลอม ส่งคิวอาร์โค้ด พร้อมเพย์ PromptPay เพื่อให้ท่านโอนเงินค่าวัคซีนเพจเฟซบุ๊กปลอม นำตราสัญลักษณ์และภาพไปใช้ โดยชื่อเพจไม่ใช่ของโรงพยาบาลจุฬาภรณ์ ราชวิทยาลัยจุฬาภรณ์ราชวิทยาลัยจุฬาภรณ์มีเฟซบุ๊ค 2 เพจเท่านั้นที่ใช้สื่อสารเรื่องการจัดสรรวัคซีน ได้แก่1⃣ โรงพยาบาลจุฬาภรณ์ ราชวิทยาลัยจุฬาภรณ์ (โรงพยาบาลจุฬาภรณ์ ราชวิทยาลัยจุฬาภรณ์)ผู้ติดตาม 662kและ2⃣ ศูนย์ข้อมูลราชวิทยาลัยจุฬาภรณ์ (ศูนย์ข้อมูลราชวิทยาลัยจุฬาภรณ์)ผู้ติดตาม 155kบัญชีไลน์ปลอม ส่งคิวอาร์โค้ด พร้อมเพย์ PromptPay ให้โอนเงินราชวิทยาลัยจุฬาภรณ์ มีช่องทาง LINE Official account: @ChulabhornhospitalAdd Friend > http://nav.cx/8DqLuQm มีผู้ติดตามเป็น Friends 2.6M สำหรับใช้สื่อสารบริการและประกาศประชาสัมพันธ์ต่างๆเท่านั้น ไม่มีบริการตอบคำถามประชาชนในไลน์บัญชีนี้ทั้งนี้ ไม่มีบริการส่ง Link โอนเงินค่าวัคซีนซิโนฟาร์ม หรือวัคซีนโควิด19 ทุกชนิดผ่านไลน์ !!! และไม่มีการจัดตั้งบัญชี LINE ศูนย์ข้อมูลราชวิทยาลัยจุฬาภรณ์ เพื่อบริการตอบคำถามประชาชนอีเมลปลอม ส่งคิวอาร์โค้ด พร้อมเพย์ PromptPay ให้โอนเงินองค์กร/นิติบุคคล ที่ได้รับการจัดสรรวัคซีนสามารถตรวจสอบวิธีการโอนเงิน และพิมพ์ใบนำฝากเงิน ผ่านการล็อคอินเข้าสู่ระบบ “ลงทะเบียนองค์กรผู้ได้รับการจัดสรรวัคซีน” ของราชวิทยาลัยจุฬาภรณ์ เท่านั้น เพื่อโอนเงินผ่านช่องทางต่างๆที่ราชวิทยาลัยจุฬาภรณ์กำหนดไม่มีการโอนเงินผ่านพร้อมเพย์ประกาศ ณ วันที่ 18 กันยายน 2564','การจัดสรรวัคซีนสามารถตรวจสอบวิธีการโอนเงิน และพิมพ์ใบนำฝากเงิน ผ่านการล็อกอินเข้าสู่ระบบ “ลงทะเบียนองค์กรผู้ได้รับการจัดสรรวัคซีน” ของราชวิทยาลัยจุฬาภรณ์เท่านั้น เพื่อโอนเงินผ่านช่องทางต่างๆ ที่ราชวิทยาลัยจุฬาภรณ์กำหนด โดยไม่มีการโอนเงินผ่านพร้อมเพย์...','ราชวิทยาลัยจุฬาภรณ์เตือนภัยมิจฉาชีพ หลอกโอนเงินค่าวัคซีน','ราชวิทยาลัยจุฬาภรณ์ ได้โพสต์เตือนภัยมิจฉาชีพ หลอกโอนเงินค่าวัคซีน แฉวิธีลวงเหยื่อ สร้างเฟซบุ๊ก บัญชีไลน์ หรือ อีเมล ปลอม ส่งคิวอาร์โค้ด พร้อมเพย์']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['ประชาชนที่ได้รับความเสียหายหรือเกิดอาการไม่พึงประสงค์หลังรับวัคซีนโควิด-19 ที่ภาครัฐจัดให้ สามารถยื่นคำร้องขอรับเงินช่วยเหลือได้ที่ รพ. ที่ฉีดสำนักงานสาธารณสุขจังหวัด หรือ สปสช. พื้นที่ ทั้ง 13 เขตทั่วประเทศ ภายใน 2 ปีนับตั้งแต่วันที่ทราบความเสียหาย 1.กรณีเสียชีวิตหรือทุพพลภาพถาวร ช่วยเหลือไม่เกิน 400,000 บาท 2.กรณีสูญเสียอวัยวะหรือพิการ ช่วยเหลือไม่เกิน 240,000 บาท 3.กรณีเกิดภาวะเจ็บป่วยที่ต้องรับการรักษาไม่เกิน 1 แสนบาท โดยเตรียมเอกสาร 1.สำเนาบัตรประชาชน 2.สำเนาใบมรณบัตร (กรณีเสียชีวิต) 3.ความเห็นแพทย์ผู้ให้การรักษา พร้อมแนบแบบคำร้อง ดาวน์โหลดที่ https://www.nhso.go.th/storage/files/841⋯dsLD-d-aETThbvoZMhUwbsijTsXLgSjv_icv-Fos สอบถามเพิ่มเติม สายด่วน สปสช. 1330 #TPchannel #วิทยุและโทรทัศน์รัฐสภา','ได้รับความเสียหาย (แพ้วัคซีน) หรือเกิดอาการไม่พึงประสงค์หลังได้รับการ "ฉีดวัคซีนโควิด-19" ที่ภาครัฐจัดให้ สามารถยื่นคำร้องขอรับเงินช่วยเหลือได้ที่โรงพยาบาล ที่ฉีดสำนักงานสาธารณสุขจังหวัด หรือ สปสช. พื้นที่ทั้ง 13 เขตทั่วประเทศ ภายใน 2 ปีนับตั้งแต่วันที่ทราบความเสียหาย','"ฉีดวัคซีน" แล้วแพ้ ดาวน์โหลดเอกสารยื่นคำร้องรับเงินที่นี่ เช็คเงื่อนไข','ฉีดวัคซีน" แล้วแพ้ ดาวน์โหลดเอกสารยื่นคำร้องรับเงินที่นี่ เช็คเงื่อนไข โดยสามารถยื่นคำร้องขอรับเงินช่วยเหลือได้ที่โรงพยาบาล ที่ฉีดสำนักงานสาธารณสุขจังหวัด หรือ สปสช. พื้นที่ทั้ง 13 เขตทั่วประเทศ']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['ไม่ควรเอา น้ำผึ้ง แช่ในตู้เย็น เพราะจะทำให้น้ำผึ้งตกผลึกและข้นเร็วขึ้น จริงหรือ','ปัจจัยที่มีผลต่อการเกิดผลึกในน้ำผึ้ง ดังนี้ อุณหภูมิในการเก็บรักษา หากเก็บรักษาที่อุณหภูมิต่ำ ให้เกิดการตกผลึกได้ ถ้าเก็บในที่อุณหภูมิต่ำ (ตู้เย็น) น้ำผึ้งจะมีผลึกน้ำตาลกลูโคสตกเป็นเกล็ดเล็กๆ','วิธีสังเกตน้ำผึ้งแท้','น้ำผึ้งแท้มีคุณสมบัติทางยาและมีคุณค่าทางโภชนาการมากมาย และด้วยเหตุที่น้ำผึ้งมีราคาแพงเมื่อเทียบกับน้ำตาล จึงมีคนหัวขี้โกงทำน้ำผึ้งปลอมออกมาหลอกขายเพื่อที่จะได้กำไรมากๆ ซึ่งสำหรับผู้ที่คุ้นเคยกับการกินน้ำผึ้งเป็นประจำอยู่แล้ว จะสามารถพิสูจน์ได้ง่ายๆ เพียงแค่ดูลักษณะหรือ การชิม แต่สำหรับผู้ที่ไม่คุ้นเคย หากไปเจอน้ำผึ้งปลอมประโยชน์ที่จะได้จากน้ำผึ้งก็ลดลง ถ้าไม่มั่นใจก็ลองพิสูจน์ได้ตามวิธีการดังต่อไปนี้']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['คณะรัฐมนตรี ไฟเขียวให้ต่างชาติมาซื้อที่ดินหรืออสังหาในไทย ได้แล้ว จริงหรือ','มติคณะรัฐมนตรี เห็นชอบ ออกวีซ่าประเภทผู้พำนักระยะยาว ให้ต่างชาติที่มีศักยภาพสูง อาศัยอยู่ในไทย พร้อมแก้กฎหมายถือครองที่ดินรองรับ ที่เห็นชอบหลักการมาตรการกระตุ้นเศรษฐกิจและการลงทุนโดยการดึงดูดชาวต่างชาติที่มีศักยภาพสูงสู่ประเทศไทย ในลักษณะผู้พำนักระยะยาว (long-term stay) โดย มี 4 กลุ่มเป้าหมาย ได้แก่1.กลุ่มประชากรโลกผู้มีความมั่งคั่งสูง2.กลุ่มผู้เกษียณอายุจากต่างประเทศ3.กลุ่มที่ต้องการทำงานจากประเทศไทย4.กลุ่มผู้มีทักษะเชี่ยวชาญพิเศษ','ครม.ตั้งเป้าให้ทันปี 65 แก้กฎหมายถือครองที่ดิน ดูดชาวต่างชาติศักยภาพสูงซื้อได้','รัชดา เผย มติคณะรัฐมนตรี เห็นชอบ ออกวีซ่าประเภทผู้พำนักระยะยาว ให้ต่างชาติที่มีศักยภาพสูง อาศัยอยู่ในไทย พร้อมแก้กฎหมายถือครองที่ดินรองรับ อยากทำให้ทัน ปี 2565']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['เจ้าแมลงก้นกระดก ห้ามตบ ห้ามบี้ ห้ามปัด เด็ดขาดเลยนะคะ เพราะอาจทำให้เกิดผื่นผิวหนังอักเสบขึ้นมาได้ค่ะ','ผื่นที่เกิดจากแมลงชนิดนี้ไม่ได้เกิดจากการที่มันกัดหรือต่อย แต่เกิดจากการที่ไปโดนแล้วบี้ทำให้สารเคมีในตัวที่ชื่อว่า paederin จะทำให้ผิวหนังบริเวณที่สัมผัสไหม้มักเเป็นรอยแปลกๆตามที่มือไปสัมผัส จะเห็นว่าแมลงชนิดนี้ไม่ได้อันตราย กัดต่อยแล้วถึงกับชีวิต แต่ สามารทำให้ผิวหนังไหม้พองได้ ดังนั้น ควรป้องกันไว้ก่อนจะดีที่สุด.','แมลงก้นกระดก ห้ามตบเด็ดขาด','ถ้ามาเกาะบนตัวคุณ ห้ามตบมันเด็ดขาด เพราะในตัวมันมีสารพิษชนิดเหลว เมื่อสัมผัสผิวเราจะทำให้ผิวหนังบริเวณที่สัมผัสไหม้มักเเป็นรอยแปลกๆ']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['การฉีดวัคซีน Sinopharm ให้กับเด็ก ยังไม่ผ่านการอนุมัติจาก อย. ไทย จริงหรือไม่','“อย.ยังไม่อนุญาต ต้องย้ำว่ายังไม่ได้อนุญาต ไม่ใช่ไม่อนุญาต เพราะยังต้องให้เขาส่งข้อมูลสำคัญ การวิจัยเฟส 3 มาเพิ่มเติมทั้งความปลอดภัยและประสิทธิภาพ รวมถึงปริมาณการใช้วัคซีนในเด็กเล็ก”','อย. ยังไม่อนุมัติใช้ ‘ซิโนฟาร์ม’ ในเด็ก รอบริษัทส่งผลศึกษาเฟส 3 เพิ่มเติม','อย. ยังไม่อนุมัติใช้ ‘ซิโนฟาร์ม’ ในเด็ก รอบริษัทส่งผลศึกษาเฟส 3 เพิ่มเติม รวมถึงขนาดการใช้วัคซีนและระยะห่างในการฉีดวัคซีน']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['หลังฉีดวัคซันโควิด จำเป็นต้องตรวจดูปริมาณภูมิคุ้มกันหลังฉีดวัคซีน หรือไม่','ปัจจุบันนี้พบว่า ส่วนใหญ่หรือมากกว่า 90% ของผู้ฉีดวัคซีนโควิดจะมีภูมิคุ้มกัน อีกทั้งยังไม่มีข้อมูลบ่งชี้ถึงระดับภูมิคุ้มกันที่สัมพันธ์กับการป้องกันการติดโรคโควิด-19 (Correlates of protection) ดังนั้นการตรวจภูมิคุ้มกันหลังรับวัคซีน ณ ปัจจุบันจึงเป็นเรื่องของความสบายใจของผู้เข้ารับการตรวจภูมิที่มีทุนทรัพย์เพียงพอ','ฉีดวัคซีนโควิด-19 แล้วจำเป็นต้องตรวจภูมิคุ้มกันไหม? ฟังคำอธิบายชัดๆ จาก ‘สหเวชศาสตร์ ธรรมศาสตร์’','ท่ามกลางความสับสนและความไม่มั่นใจ เกิดเป็นคำถามดังขึ้นให้ได้ยินอย่างหนาหูว่า ที่สุดแล้ว จำเป็นต้อง “ตรวจภูมิ” หลังฉีดวัคซีนโควิด-19 หรือไม่ ?']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['เห็นกันว่าพืชกระท่อมนั้นถูกเปลี่ยนให้ถูกกฎหมายแล้วนำมาต้มได้ส่วนผมสงสัยว่าแล้วถ้าเรานำมาต้มแล้วผสมกับยาแก้ไอแล้วจะผิดกฎหมายเปล่าครับ','ปัจจุบันพืชกระท่อมพ้นจากบัญชียาเสพติดประเภท 5 แล้ว และสามารถนำมาใช้ในวิถีชีวิตท้องถิ่น ทั้งการเคี้ยวใบ การปลูก การครอบครองและการขายใบสดที่ไม่ได้ปรุงหรือทำเป็นอาหารได้อย่างเสรีไม่ผิดกฎหมาย กำหนดให้อาหารที่ปรุงจากพืชกระท่อมเป็นอาหารที่ห้ามผลิต นำเข้าหรือจำหน่ายหรือแม้กระทั่งน้ำต้มกระท่อมที่ไม่ได้ผสมกับสิ่งใดเลยก็เป็นสิ่งที่ห้ามผลิตเพื่อจำหน่ายตามประกาศฉบับนี้ หากการฝ่าฝืน ผลิต และขายอาหาร ที่ พ.ร.บ.อาหาร ห้าม มีโทษตามมาตรา','ยังไม่ปลดล็อก! เตือน ‘ต้มน้ำท่อม-ทำอาหาร-ผสมสมุนไพร’ ยังผิดกฎหมาย','"สมศักดิ์" เตือนต้มน้ำท่อม-ทำอาหาร-ผสมสมุนไพร ยังผิดก.ม. แนะ สธ.ปลดล็อกกฎหมายแปรรูปกระท่อมผสมอาหาร-ทำผลิตภัณฑ์สมุนไพรจำหน่าย ชี้เป็นอุปสรรคการค้าวิถีชาวบ้าน']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['กินไข่ต้ม สามารถช่วยเรื่องการลลดน้ำหนักได้ โดยผลการวิจัยจากศูนย์ Rochester Centre for Obesity in America และมหาวิทยาลัยลุยเซียนา ในประเทศสหรัฐอเมริกา ที่ได้ออกมาทำการวิจัยพร้อมเผยว่า กินไข่เป็นอาหารเช้าสามารถช่วย จำกัด ปริมาณแคลอรี่ของคุณตลอดทั้งวัน ได้มากกว่า 400 กิโลแคลอรี','รับประทานอาหารเช้าที่มีไข่เป็นส่วนประกอบ หรืออาหารเช้าแบบเบเกิล ซึ่งมี กิโลแคลอรี ในปริมาณเท่ากันและระดับโปรตีนเกือบเท่ากัน นักวิจัยได้บันทึกพฤติกรรมการกินของผู้หญิงและพบว่าก่อนอาหารกลางวันผู้หญิงที่กินไข่เป็นอาหารเช้ารู้สึกหิวน้อยลง','"ไข่ต้ม"ทานแล้ว ช่วยลดน้ำหนักได้ มีผลวิจัย','เรื่องสุขภาพถือเป็นเรื่องสำคัญที่มองข้ามไม่ได้ หลายคนมองหาวิธีลดความอ้วน ผลการวิจัยในประเทศสหรัฐอเมริกาออกมาเผยว่า กินไข่เป็นอาหารเช้าสามารถช่วย จำกัด ปริมาณแคลอรี่ของคุณตลอดทั้งวันได้']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['คณะรัฐมนตรี อนุมัติหลักการให้บริษัทหรือห้างหุ้นส่วนนิติบุคคลนำรายจ่ายค่าซื้อชุดตรวจโควิด-19 แบบเร่งด่วน (Antigen Test Kit) สำหรับพนักงานหรือลูกจ้าง มาหักเป็นรายจ่ายได้ 1.5 เท่า มีผลจนถึง 31 มีนาคม 2565 เพื่อให้ผู้ประกอบการจัดหาชุดตรวจ ATK ให้พนักงานหรือลูกจ้างใช้ตรวจหาเชื้อโควิด-19 ตามมาตรการที่กระทรวงสาธารณสุขกำหนด','ครม.ไฟเขียว สรรพากรออกมาตรการภาษีเพื่อสนับสนุนรายจ่ายค่าชื้อชุดตรวจโควิด ให้บริษัทหรือห้างหุ้นส่วนนิติบุคคล หักรายจ่ายได้ 1.5 เท่า กรณีซื้อชุดตรวจ ATK ตรวจโควิดพนักงาน ถึง 31 มี.ค. 65','สรรพากรให้บริษัทซื้อชุดตรวจ ATK หักรายจ่ายได้ 1.5 เท่า','ครม.ไฟเขียว สรรพากรออกมาตรการภาษีเพื่อสนับสนุนรายจ่ายค่าชื้อชุดตรวจโควิด ให้บริษัทหรือห้างหุ้นส่วนนิติบุคคล หักรายจ่ายได้ 1.5 เท่า กรณีซื้อชุดตรวจ ATK ตรวจโควิดพนักงาน ถึง 31 มี.ค. 65']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['แอปเป๋าตังมีดีกว่าที่คิด ไม่น่าเชื่อว่าในเป๋าตัง ก็มีเรื่องสุขภาพใกล้ตัวทุกเพศวัยตั้งแต่แรกเกิด จนถึงผู้สูงอายุ เช่นบริการปรึกษาโรคซึมเศร้าและตรวจสอบความสามารถในการช่วยเหลือตนเองมีบริการสำหรับผู้หญิง เช่น สิทธิตรวจคัดกรองมะเร็งปากมดลูก และสิทธิฝากครรภ์ใครมีแอปเป๋าตังแล้ว ลองกดเข้าไปปรึกษาเรื่องสุขภาพได้','“กระเป๋าสุขภาพ หรือ Health Wallet” ให้บริการสร้างเสริมสุขภาพและป้องกันโรคก่อนเกิดการรักษาแก่คนไทยทุกคนฟรี! สามารถเช็คสิทธิประโยชน์ของตัวเองได้ มีการเก็บข้อมูลประวัติการรักษาจะเชื่อมโยงกับสำนักงานหลักประกันสุขภาพถ้วนหน้า สามารถเช็คสถานพยาบาลที่ใกล้บ้านหรือสะดวกเพื่อทำนัดพบหมอ ฉีดวัคซีน ตรวจคัดกรองโรคต่างๆ','แอปฯ เป๋าตัง เปิดฟีเจอร์ ตรวจสอบสิทธิ-นัดหมอ “โรคซึมเศร้าในผู้สูงวัย” ผ่านวอลเล็ต “กระเป๋าสุขภาพ”','เพราะการป้องกันย่อมดีกว่าการรักษา...ระบบสาธารณสุขทั่วโลกจึงใส่ใจและให้ความสำคัญกับการป้องกันและสร้างเสริมความรู้ด้านสุขภาพให้แก่ประชาชนไม่น้อยไปกว่าการค้นหาวิธีการรักษาโรค']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['ประเทศออสเตรเลีย จะเปิดประเทศรับนักท่องเที่ยว ในเดือนพฤศจิกายน 2554 นี้','คาดว่าผู้ใช้เขียนปี พ.ศ. ผิด เป็นปี 2554 โดยจริงแล้วเป็น พฤศจิกายน 2564รัฐบาลออสเตรเลียพร้อมเเปิดประเทศอย่างปลอดภัย เพื่อให้พลเมืองออสเตรเลียในต่างประเทศกลับบ้านเกิดได้ รวมทั้งอนุญาตให้นักท่องเที่ยวเดินทางเข้าออสเตรเลียได้อีกครั้ง โดยแผนการเปิดประเทศออสเตรเลีย เบื้องต้นจะมีการประเมินความพร้อมขึ้นอยู่กับอัตราการฉีดวัคซีนในแต่ละพื้นที่ แต่คาดว่าน่าจะเริ่มได้ในเดือน พ.ย. นี้ มุ่งไปที่ชาวออสเตรเลียที่ได้รับการฉีดวัคซีนแล้วและผู้มีที่พำนักถาวรในออสเตรเลียที่จะได้รับอนุญาตให้เดินทางเข้ามาในประเทศออสเตรเลียได้ รัฐบาลออสเตรเลียจะยังคงมีการกำหนดจำนวนผู้ที่จะได้เดินทางเข้าประเทศ รวมทั้งเมื่อเข้ามาแล้วจะต้องกักตัวเป็นเวลา 14 วัน','ออสเตรเลียเตรียมเปิดพรมแดน พ.ย. 64 หลังล็อกสกัดโควิดนาน 18 เดือน','รัฐบาลออสเตรเลียเตรียมเปิดพรมแดนระหว่างประเทศ ในเดือน พ.ย. อนุญาตให้พลเมืองออสเตรเลียเดินทางไปต่างประเทศได้แล้ว หลังล็อกสกัดโควิดมานานถึง 18 เดือน']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['"เด็กที่ไม่ได้ฉีดวัคซีนก็สามารถเข้าเรียนได้ ควบคู่กับการเข้มงวดมาตรการป้องกันโรค ภายใต้การพิจารณาของคณะกรรมการโรคติดต่อจังหวัด"สาธิต ปิตุเตชะ รมช.สธ.','เด็กที่ไม่ได้ฉีดวัคซีนก็สามารถเข้าเรียนได้ ควบคู่กับการเข้มงวดมาตรการป้องกันโรค ภายใต้การพิจารณาของคณะกรรมการโรคติดต่อจังหวัด สำหรับเด็กที่มีโรคประจำตัวไม่ได้เรียนหนังสือ หรือกลุ่มนอกระบบการศึกษา สามารถเข้ารับวัคซีนที่โรงพยาบาลที่รักษาประจำได้ ส่วนเด็กเรียนที่บ้านหรือโฮมสคูล ลงทะเบียนรับวัคซีนกับโรงพยาบาลใกล้บ้านได้เช่นกัน','นายกฯ Kick off ฉีดวัคซีนไฟเซอร์นักเรียนอายุ 12–18 ปี','วันที่ 4 ก.ย. 2564 พล.อ.ประยุทธ์ จันทร์โอชา นายกรัฐมนตรี เป็นประธานเปิดงาน Kick off สร้างเกราะป้องกันด้วยวัคซีนเด็กปลอดภัย เรียนอุ่นใจ ต้อนรับเปิดเทอม เพื่อสร้างความเชื่อมั่นในการฉีดวัคซีนในเด็กกับพ่อแม่ ผู้ปกครอง พร้อมชูมาตรการ Sandbox ลดความเสี่ยงของโควิด-19 ณ โรงเรียนพิบูลย์อุปถัมภ์ กรุงเทพฯ']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['หมอระบบประสาทบอกว่า ผลไม่ต่างจากระบบฝังเข็ม มือซ้ายจับหูขวา มือขวาจับหูซ้าย กระตุ้นการทำงานสมองซีกซ้ายและซีกขวาดีสำหรับคนปกติทุกเพศ ทุกวัย คนขี้หลงขี้ลืม Alzheimer แม้แต่ Autistic ก็จะค่อยๆดีขึ้นมือซ้ายจับติ่งหูขวามือขวาจับติ่งหูซ้ายย่อตัวลงหายใจเข้ายืดตัวขึ้นหายใจออกหมอเรียกวิธีนี้ว่า Superbrain Yogaทำวันละ 3 นาทีง่ายมั้ยล่ะ','ตามที่มีการเผยแพร่ในสื่อต่าง ๆ เกี่ยวกับประเด็น ท่าบริหาร SuperBrain Yoga ช่วยกระตุ้นการทำงานของสมองซีกซ้ายและซีกขวาได้ ทางศูนย์ต่อต้านข่าวปลอมได้ดำเนินการตรวจสอบข้อเท็จจริงโดย สถาบันประสาทวิทยา พบว่าประเด็นดังกล่าวนั้น เป็นข้อมูลบิดเบือน','ข่าวบิดเบือน ท่าบริหาร SuperBrain Yoga ช่วยกระตุ้นการทำงานของสมองซีกซ้ายและซีกขวาได้','ตามที่มีการเผยแพร่ในสื่อต่าง ๆ เกี่ยวกับประเด็น ท่าบริหาร SuperBrain Yoga ช่วยกระตุ้นการทำงานของสมองซีกซ้ายและซีกขวาได้ ทางศูนย์ต่อต้านข่าวปลอมได้ดำเนินการตรวจสอบข้อเท็จจริงโดย สถาบันประสาทวิทยา พบว่าประเด็นดังกล่าวนั้น เป็นข้อมูลบิดเบือน']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['การเดินทางไปต่างประเทศ ต้องแสดงวัคซีน พาสปอร์ต ที่ออกโดยรัฐ เท่านั้น','ผู้ที่ฉีดวัคซีนโควิด-19 แล้ว หายป่วยจากโรคโควิด หรือมีผลตรวจโควิดเป็นลบ เพื่อที่จะสามารถเดินทางท่องเที่ยวในยุโรปได้ส่วน สหราชอาณาจักรเอง ก็กำลังจะออกวัคซีนพาสปอร์ตเช่นกัน เพื่อใช้ในการเดินทางท่องเที่ยวในอังกฤษ ยุโรป และ ประเทศที่อังกฤษอนุญาตให้เดินทางไปกลับได้โดยไม่ต้องกักตัว','"วัคซีนพาสปอร์ต" บัตรผ่านเดินทางในต่างประเทศ เงื่อนไข ขั้นตอนการขอไม่ยุ่งยาก','วัคซีนพาสปอร์ต ใบเบิกทาง เดินทางท่องเที่ยวในต่างประเทศขั้นตอนการเตรียมเอกสาร ยื่นขอวัคซีนพาสปอร์ต ไม่ยุ่งยากอย่างที่คิดหากใครที่ติดตามข่าว พล.อ.ประยุทธ์ จันทร์โอชา นายกรัฐมนตรี เดินทางเข้ารับการฉีดวัคซีนแอสตราเซเนกา ป้องกันโควิด-19 เข็มที่ 2 เมื่อช่วงเช้าวันนี้ (24 พ.ค.) หลังฉีดวัคซีนเข็มแรกไปเมื่อวันที่ 16 มี.ค.64 จากนั้น อธิบดีกรมควบคุมโรค ได้มอบ "วัคซีนพาสปอร์ต" ให้กับนายกฯ ใช้ในการเดินทางไปต่างประเทศ โดยแนบกับหนังสือเดินทาง เพื่อเป็นการยืนยันว่า ปราศจากเชื้อโควิด และได้รับการฉีดวัคซีนครบแล้ว 2 โดส ซึ่งบางคนคงมีคำถามว่า "วัคซีนพาสปอร์ต" คืออะไร และสามารถใช้เดินทางเข้าประเทศไหนได้บ้าง']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ["'ติดแอร์' หัวเตียงนอน ทำให้เกิด 'เชื้อราในสมอง'",'เว็บไซต์ศูนย์ต่อต้านข่าวปลอม ประเทศไทย รายงานว่า ตามที่มีการเผยแพร่ข้อมูลประเด็นเรื่องติดแอร์บนหัวเตียงนอน ก่อให้เกิดเชื้อราในสมอง ทางศูนย์ต่อต้านข่าวปลอมได้ดำเนินการตรวจสอบข้อเท็จจริงโดย สถาบันประสาทวิทยา กรมการแพทย์ กระทรวงสาธารณสุข พบว่าประเด็นดังกล่าวนั้น เป็นข้อมูลเท็จ',"ข่าวปลอม อย่าแชร์! 'ติดแอร์' หัวเตียงนอน ทำให้เกิด 'เชื้อราในสมอง'",'เช็คแล้ว ข่าวปลอม อย่าแชร์! "ติดแอร์" บนหัวเตียงนอน ก่อให้เกิด "เชื้อราในสมอง" สถาบันประสาทวิทยายัน "ข้อมูลเท็จ"']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['เด็กนักเรียนอายุ 12 ปีขึ้นไป ฉีดวัคซีนป้องกันโรคโควิด-19 หากเกิดอาการไม่พึงประสงค์หลังฉีดวัคซีน สปสช.จัดระบบช่วยเหลือเบื้องต้นดูแล ขอยื่นเรื่องรับเงินช่วยเหลือได้','หากเกิดอาการไม่พึงประสงค์หลังฉีดวัคซีนโควิด-19 ตามแผนงานโครงการที่รัฐจัดให้ฟรี สามารถยื่นเรื่องขอรับเงินช่วยเหลือเบื้องต้นกับสำนักงานหลักประกันสุขภาพแห่งชาติ (สปสช.) ได้ที่ โรงพยาบาลที่ฉีด สำนักงานสาธารณสุขจังหวัด (สสจ.) หรือที่สำนักงาน สปสช.สาขาเขตพื้นที่ทั้ง 13 เขต โดยมีระยะเวลายื่นคำร้องได้ภายใน 2 ปีนับแต่วันที่ทราบความเสียหาย เมื่อมีผู้มายื่นคำร้องแล้ว คณะอนุกรรมการฯ ระดับเขตพื้นที่จะเร่งพิจารณาให้แล้วเสร็จโดยเร็ว ซึ่งในกรณีที่ผู้ยื่นคำร้องไม่เห็นด้วยกับผลการวินิจฉัย ก็มีสิทธิยื่นอุทธรณ์ต่อเลขาธิการ สปสช. ได้ภายใน 30 วันนับแต่วันที่ทราบผลการวินิจฉัย','"นักเรียน" เช็คอาการไม่พึงประสงค์ "วัคซีนโควิด" และหลักเกณฑ์รับเยียวยา','"สปสช." ขานรับนโยบายรัฐบาลฉีด "วัคซีนโควิด-19" ให้กับเด็กนักเรียนอายุ 12 ปีขึ้นไป หากเกิด "อาการไม่พึงประสงค์" หลังฉีดวัคซีน สปสช. จัดระบบช่วยเหลือเบื้องต้นดูแล ขอยื่นเรื่องรับเงินช่วยเหลือได้ ย้ำรายงานผู้เกิดผลข้างเคียงมีไม่มาก']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['มีการส่งต่อในกลุ่มไลน์ให้พึงระวัง ผู้ไม่ประสงค์ดีนำเอาตะปูไปใส่ไว้ในลูกส้มแป้น','อ.เจษฎา นักวิชาการอิสระ แจงชัด แชร์เตือนภัย ‘ส้มตะปู’ แท้จริงเกิดในอินโดฯวันนี้ (12 ม.ค.) ล่าสุด อ.เจษฎา เด่นดวงบริพันธ์ ผู้ช่วยศาสตราจารย์ภาควิชาชีววิทยา คณะวิทยาศาสตร์ จุฬาลงกรณ์มหาวิทยาลัย ได้ออกมาแสดงความเห็นต่อเรื่องดังกล่าวว่า“เรื่องส้มสอดไส้ตะปู เป็นเรื่องในอินโดนีเซีย”','เฉลยแล้ว! แชร์เตือนภัย ‘ส้มตะปู’ แท้จริงเกิดในอินโดฯ','อ.เจษฎา นักวิชาการอิสระ แจงชัด แชร์เตือนภัย ‘ส้มตะปู’ แท้จริงเกิดในอินโดฯ วันนี้ (12 ม.ค.) ล่าสุด อ.เจษฎา เด่นดวงบริพันธ์ ผู้ช่วยศาสตราจารย์ภาควิชาชีววิทยา คณะวิทยาศาสตร์ จุฬาลงกรณ์มหาวิทยาลัย ได้ออกมาแสดงความเห็นต่อเรื่องดังกล่าวว่า“เรื่องส้มสอดไส้ตะปู เป็นเรื่องในอินโดนีเซีย”กะแล้วว่าเป็นแค่ข่าวลือ ไม่ใช่เรื่องจริงในไทย เห็นมีแต่รูปกับลือกันว่าโดนกันแถวประจวบฯ ไม่เห็นมีข่าวจริง ๆ เป็นทางการว่าใครโดนไปบ้าง แต่ยังไงก็ระวัง ๆ กันไว้ก็ดีครับ แชร์กันกระหน่ำขนาดนี้ มิจฉาชีพได้ไอเดียทำมาหากินเพิ่มขึ้น ปล. แล้วเวลาจะขับรถหลบส้มหรือของอะไรที่หล่นพื้นอยู่ ดูรถข้าง ๆ ด้วยนะครับ เดี๋ยวพลาดชนกันอีก']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['เด็กนักเรียนอายุ 12 ปีขึ้นไป ฉีดวัคซีนป้องกันโรคโควิด-19 หากเกิดอาการไม่พึงประสงค์หลังฉีดวัคซีน สปสช.จัดระบบช่วยเหลือเบื้องต้นดูแล ขอยื่นเรื่องรับเงินช่วยเหลือได้','ตามที่มีการนำเสนอข่าว เกี่ยวกับประเด็นเรื่อง นักเรียนที่ได้รับผลข้างเคียงหลังฉีดวัคซีนโควิด-19 สามารถยื่นเรื่องรับเงินช่วยเหลือได้ สูงสุดไม่เกิน 4 แสนบาท ทางศูนย์ต่อต้านข่าวปลอมได้ดำเนินการตรวจสอบข้อเท็จจริงโดย สำนักงานหลักประกันสุขภาพแห่งชาติ กระทรวงสาธารณสุข พบว่าประเด็นดังกล่าวนั้น เป็นข้อมูลจริง','นักเรียนที่ได้รับผลข้างเคียงหลังฉีดวัคซีนโควิด-19 สามารถยื่นเรื่องรับเงินช่วยเหลือได้ สูงสุดไม่เกิน 4 แสนบาท จริงหรือ?','ตามที่มีการนำเสนอข่าว เกี่ยวกับประเด็นเรื่อง นักเรียนที่ได้รับผลข้างเคียงหลังฉีดวัคซีนโควิด-19 สามารถยื่นเรื่องรับเงินช่วยเหลือได้ สูงสุดไม่เกิน 4 แสนบาท ทางศูนย์ต่อต้านข่าวปลอมได้ดำเนินการตรวจสอบข้อเท็จจริงโดย สำนักงานหลักประกันสุขภาพแห่งชาติ กระทรวงสาธารณสุข พบว่าประเด็นดังกล่าวนั้น เป็นข้อมูลจริง']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['ทางรัฐมีการเรียกเงินคืนจากโครงการต่างๆ เช่นโครงการคนละครึ่ง โครงการเราชนะ ซึ่งโดนทั้งร้านค้าและลูกค้าที่แสกน','พ่อค้าแม่ค้าหลายรายที่เข้าร่วมโครงการรัฐไม่ว่าจะเป็น โครงการเราชนะ โครงการ ม.33 รวมถึงโครงการคนละครึ่ง ได้รับหนังสือแจ้งจากการกระทรวงการคลังว่าต้องทำการคืนเงินให้กับรัฐบาล เนื่องจากไม่ได้ปฏิบัติตามเงื่อนไข โดยผู้ที่ได้รับหนังสือดังกล่าวสามารถติดต่อกับกระทรวงการคลังเพื่อขออุทธรณ์ และชี้แจงข้อเท็จจริง','ร้านค้า"เราชนะ-คนละครึ่ง"โวยถูกเรียกเงินคืนเหตุผิดเงื่อนไข"แลกเงินสด"','ร้านค้าโครงการรัฐ"เราชนะ-คนละครึ่ง" โวยถูก"สศค."เรียกเงินคืนอ้างเหตุไม่ปฏิบัติตามเงื่อนไข ส่วนใหญ่เป็นร้านรับแลกเงินสด- ออนไลน์ สูงสุดเรียกคืน 17 ล้านบาท ด้านนายสามารถ อดีตผช.รมต.ยุติธรรม เตรียมนำผู้ประกอบการร้องทุกข์ต่อนายกฯที่ทำเนียบ 12 ต.ค. นี้']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

clean_text = ['นักวิทย์พบโรคติดเชื้ออุบัติใหม่ซุ่มเงียบในญี่ปุ่น สามารถแพร่สู่มนุษย์','คณะนักวิทยาศาสตร์ญี่ปุ่น ค้นพบไวรัสตัวหนึ่งที่ไม่เป็นที่รู้จักมาก่อน ซึ่งมีความสามารถแพร่เชื้อสู่มนุษย์ และจากการศึกษายังชี้ว่า ไวรัสอุบัติใหม่ซึ่งเรียกชื่อกันว่า "ไวรัสเยโซ (Yezo virus)" ตัวนี้ มีความสัมพันธ์กับเชื้อโรคที่ก่อให้เกิดไข้เลือดออกไครเมียน-คองโก (Crimean-Congo hemorrhagic fever) และเชื้อโรคแกะไนโรบี (Nairobi sheep)','นักวิทย์พบโรคติดเชื้ออุบัติใหม่ซุ่มเงียบในญี่ปุ่น เกิดจากถูก‘เห็บ’กัดแล้วแพร่ไวรัสร้ายเข้าสู่มนุษย์','คณะนักวิทยาศาสตร์ญี่ปุ่น ค้นพบไวรัสตัวหนึ่งที่ไม่เป็นที่รู้จักมาก่อน ซึ่งมีความสามารถแพร่เชื้อสู่มนุษย์ และจากการศึกษายังชี้ว่า ไวรัสอุบัติใหม่ซึ่งเรียกชื่อกันว่า "ไวรัสเยโซ (Yezo virus)" ตัวนี้ มีความสัมพันธ์กับเชื้อโรคที่ก่อให้เกิดไข้เลือดออกไครเมียน-คองโก (Crimean-Congo hemorrhagic fever) และเชื้อโรคแกะไนโรบี (Nairobi sheep)']
original_text_and_headline_news.append(clean_text[0])
original_text_and_headline_news.append(clean_text[2])

# rows, cols = (len(original_text_and_headline_news), len(original_text_and_headline_news))
# similarity_matrix = [[0]*cols]*rows

In [35]:
print(original_text_and_headline_news) #เลขคู่คือข้อความดั้งเดิม เลขคี่ถัดกัน(เลขคู่+1)คือหัวข้อข่าวนั้นๆ

['น้ำจะท่วมกรุงเทพฯ ในเดือนตุลา 64', 'น้ำท่วมจะไปไหน ข่าวลือเริ่มแรง กรุงเทพฯ-นนทบุรี จะจมน้ำ?', 'ขณะนี้ คิวในการยื่นขอ VISA ประเทศสหรัฐอเมริกา คิวอยู่ที่เดือนสิงหาคม พ.ศ. 2565 แล้ว จริงหรือไม่', 'ระยะเวลารอคิวการยื่นวีซ่าสามารถเช็กได้ที่เว็บไซต์https://travel.state.gov/content/travel/⋯sa-information-resources/wait-times.htmlจากการเช็กล่าสุดพบว่าระยะเวลาการรอวีซ่าคือVisitor Visa 35 Calendar DaysStudent/Exchange Visitor Visas 2 Calendar DaysAll Other Nonimmigrant Visas 2 Calendar Daysไม่ถึงปี 65 แต่อย่างใด', 'ประเทศไทย มีจังหวัดเพิ่มอีก 6 จังหวัด จริงหรือ', 'มท.ยืนยันประเทศไทยมีแค่ 76 จังหวัด', 'ยะลา คุมเข้มมาตรการ เคอร์ฟิว - ร้านขายอาหาร เครื่องดื่ม ปิด 2 ทุ่ม', 'ยะลา คุมเข้มมาตรการ เคอร์ฟิว - ร้านขายอาหาร เครื่องดื่ม ปิด 2 ทุ่ม ลดเสี่ยงโควิด-19', 'ถ้าใครมีเส้นเลือดในสมองเริ่มตีบแล้ว จะไม่สามารถยืนด้วยขาข้างเดียวถึง 20 วินาที ซึ่งน่าจะจริง เพราะ ได้ไปทำ MRI ที่สมองแล้ว แพทย์พบว่า มีเส้นเลือดในสมองตีบ และ ได้มาพบคลิบนี้ ลองยืนขาเดียว ปรากฏว่า ยืนได้เพียง 3 วินาทีเท่านั้นจึงคิดว่า มีความ

In [36]:
print('tokenized text:\n',split_word(original_text_and_headline_news[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(original_text_and_headline_news[1])) #หัวข่าวอ้างอิง

tokenized text:
 ['น้ำ', 'ท่วม', 'กรุงเทพฯ', 'เดือน', 'ตุลา']
tokenized text:
 ['น้ำ', 'ท่วม', 'ข่าว', 'ลือ', 'แรง', 'กรุงเทพฯ', 'นนทบุรี', 'จม', 'น้ำ']


In [37]:
original_and_headline_tokens_list = [split_word(txt) for txt in original_text_and_headline_news]

In [38]:
original_and_headline_tokens_list

[['น้ำ', 'ท่วม', 'กรุงเทพฯ', 'เดือน', 'ตุลา'],
 ['น้ำ', 'ท่วม', 'ข่าว', 'ลือ', 'แรง', 'กรุงเทพฯ', 'นนทบุรี', 'จม', 'น้ำ'],
 ['คิว',
  'ยื่น',
  'VISA',
  'ประเทศ',
  'สหรัฐอเมริกา',
  'คิว',
  'เดือน',
  'สิงหาคม',
  'พ',
  'ศ'],
 ['เวลา',
  'รอ',
  'คิว',
  'ยื่น',
  'วีซ่า',
  'สา',
  'รถ',
  'เช็ก',
  'เว็บไซต์',
  'https',
  'travel.state.gov',
  'content',
  'travel',
  '⋯',
  'sa',
  'information',
  'resources',
  'wait',
  'times.html',
  'จา',
  'กการเช็',
  'กล่า',
  'เวลา',
  'รอ',
  'วีซ่า',
  'Visitor',
  'Visa',
  'Calendar',
  'DaysStudent',
  'Exchange',
  'Visitor',
  'Visas',
  'Calendar',
  'DaysAll',
  'Other',
  'Nonimmigrant',
  'Visas',
  'Calendar',
  'Days',
  'ปี',
  'ใด'],
 ['ประเทศไทย', 'จังหวัด', 'จังหวัด'],
 ['มท', 'ประเทศไทย', 'จังหวัด'],
 ['ยะลา',
  'คุม',
  'เข้ม',
  'มาตรการ',
  'เคอร์ฟิว',
  'ร้าน',
  'ขาย',
  'อาหาร',
  'เครื่อง',
  'ดื่ม',
  'ทุ่ม'],
 ['ยะลา',
  'คุม',
  'เข้ม',
  'มาตรการ',
  'เคอร์ฟิว',
  'ร้าน',
  'ขาย',
  'อาหาร',
  'เครื่อง',
 

In [39]:
tokens_list_j = [','.join(tkn) for tkn in original_and_headline_tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
original_c_feat = cvec.fit_transform(tokens_list_j)

In [40]:
original_text_and_headline_news_vector = original_c_feat[:,:].todense()
print(original_text_and_headline_news_vector)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [41]:
original_text_vector = original_text_and_headline_news_vector[::2]
headline_news_vector = original_text_and_headline_news_vector[1::2]

In [42]:
len(original_text_vector)

34

In [43]:
len(headline_news_vector)

34

In [44]:
find_cosine_similarity(original_text_vector, headline_news_vector, 0,0)

0.5393598899705937

In [45]:
def create_similarity_matrix(original_text_vector, headline_news_vector):
    similarity_matrix = []

    for text in range(len(original_text_vector)):
        tmp_array = []
        for headline in range(len(headline_news_vector)):
            tmp_array.append(find_cosine_similarity(original_text_vector, headline_news_vector, text, headline))
        similarity_matrix.append(tmp_array)
    
    df = pd.DataFrame(similarity_matrix, columns =[i for i in range(len(original_text_vector))], dtype = float)
    return df

In [46]:
df_similarity_matrix = create_similarity_matrix(original_text_vector, headline_news_vector)

In [47]:
cm = sns.light_palette("green", as_cmap=True)
df_similarity_matrix.style.background_gradient(cmap=cm)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,0.539360,0.000000,0.000000,0.000000,0.000000,0.000000,0.124035,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.223607,0.000000,0.000000,0.000000,0.000000,0.134840,0.169031,0.000000,0.000000,0.141421,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.114708,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.091287,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.182574,0.000000,0.000000,0.091287,0.000000,0.000000,0.000000,0.087039,0.000000,0.000000
2,0.000000,0.000000,0.774597,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.186501,0.282843,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.856349,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.095346,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.481932,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038292,0.000000,0.000000,0.000000,0.000000,0.000000,0.131165,0.000000,0.146647,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.013518,0.000000,0.000000,0.000000,0.463713,0.000000,0.000000,0.000000,0.000000,0.042563,0.032275,0.030773,0.000000,0.000000,0.000000,0.030773,0.000000,0.030773,0.000000,0.000000,0.038576,0.000000,0.000000,0.000000,0.029463,0.000000,0.000000,0.000000,0.000000,0.000000,0.030773,0.000000,0.027277
6,0.018280,0.022083,0.000000,0.046961,0.052723,0.028356,0.491835,0.064416,0.009140,0.004570,0.050566,0.004793,0.086829,0.008102,0.004793,0.037892,0.036559,0.006188,0.018280,0.028356,0.091399,0.040101,0.015157,0.025007,0.129410,0.035003,0.000000,0.009586,0.004375,0.009586,0.005359,0.036559,0.013710,0.024305
7,0.031392,0.029879,0.000000,0.152335,0.054874,0.004638,0.028877,0.373082,0.036624,0.204049,0.028946,0.027437,0.020928,0.013913,0.005487,0.000000,0.020928,0.000000,0.015696,0.148406,0.000000,0.013117,0.000000,0.016360,0.192059,0.030056,0.031363,0.043899,0.150278,0.186571,0.006135,0.177889,0.005232,0.282899
8,0.000000,0.000000,0.000000,0.091287,0.000000,0.000000,0.000000,0.000000,0.533002,0.000000,0.000000,0.000000,0.000000,0.094491,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.102062,0.000000,0.111803,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.012610,0.000000,0.073743,0.050176,0.008481,0.017603,0.150737,0.028705,0.287045,0.033085,0.000000,0.019136,0.144182,0.180633,0.000000,0.000000,0.000000,0.038273,0.203551,0.047841,0.023989,0.021156,0.029919,0.080281,0.174056,0.024581,0.150528,0.009161,0.270950,0.011220,0.258341,0.009568,0.084813


In [48]:
original_text_and_headline_news_vector.shape[1]

1120

In [49]:
def create_dataset(original_text_vector, headline_news_vector):
    dataset = []
    for text in range(len(original_text_vector)):
        tmp = []
        for each_word in range(original_text_and_headline_news_vector.shape[1]):
            tmp.append(original_text_vector[text, each_word])
            tmp.append(headline_news_vector[text, each_word])
        tmp.append(1)
        dataset.append(tmp)

    headline = text
    for random_match in range(len(original_text_vector)):
        tmp = []
        while(text == headline):
            text = random.randint(1, len(original_text_vector))
            headline = random.randint(1, len(headline_news_vector))

        for each_word in range(original_text_and_headline_news_vector.shape[1]):
            tmp.append(original_text_vector[text, each_word])
            tmp.append(headline_news_vector[headline, each_word])
        tmp.append(0)
        dataset.append(tmp) 

    df = pd.DataFrame(dataset, columns =[i for i in range(original_text_and_headline_news_vector.shape[1]*2+1)], dtype = float)
    return df

In [50]:
df_dataset = create_dataset(original_text_vector, headline_news_vector)

In [51]:
df_dataset

,0,1,2,3,4,5,6,7,8,9,...,2231,2232,2233,2234,2235,2236,2237,2238,2239,2240
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
original_text_and_headline_news_vector.shape[1]

1120

In [53]:
X = df_dataset.drop(df_dataset.columns[(original_text_and_headline_news_vector.shape[1]*2)],axis=1)
y = df_dataset[df_dataset.columns[(original_text_and_headline_news_vector.shape[1]*2)]]

In [54]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.3, random_state = 100, stratify=y)

In [55]:
clf_svm = svm.SVC(kernel='rbf')
clf_svm.fit(X_train, y_train)

SVC()

In [56]:
y_pred = clf_svm.predict(X_test)  
y_pred

array([1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       1., 1., 0., 0.])

In [57]:
print('Non Standardize')
print('Accuracy of SVM classifier on training set: {:.4f}'
     .format(clf_svm.score(X_train, y_train)*100))
print('Accuracy of SVM classifier on test set: {:.4f}'
     .format(clf_svm.score(X_test, y_test)*100))

Non Standardize
Accuracy of SVM classifier on training set: 97.8723
Accuracy of SVM classifier on test set: 100.0000


In [58]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        11
         1.0       1.00      1.00      1.00        10

    accuracy                           1.00        21
   macro avg       1.00      1.00      1.00        21
weighted avg       1.00      1.00      1.00        21

[[11  0]
 [ 0 10]]
